In [1]:
import cbpro as gdax
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import OrderedDict

from datetime import datetime, timedelta
from math import ceil
from time import sleep

%matplotlib inline
client = gdax.PublicClient()

In [2]:
def historic_data(client, product, num_candles, granularity, max_per=300):
    now = datetime.utcnow().replace(microsecond=0).replace(second=0)
    unit = timedelta(seconds=granularity)
    begin = now - (num_candles * unit)
    delta = unit * max_per
    full = ceil(num_candles / max_per)
    rates = []
    for i in range(0,full):
        frm = begin + (delta * i)
        to  = frm + delta
        chunk = client.get_product_historic_rates(
            product, start=frm.isoformat(), end=to.isoformat(), granularity=granularity)
        if type(chunk) is not list:
            print(chunk)
        else:
            rates.extend(chunk)
            print('From: {}, To: {} - {} points'.format(frm.isoformat(), to.isoformat(), len(chunk)))
        sleep(0.6)
    return rates 

In [3]:
def save_rates(product, num_candles=None, granularity=60):
    rates = historic_data(client=client, product=product, num_candles=num_candles, granularity=granularity)
    print('{} prices'.format(len(rates)))
    # Load into Pandas
    pd_rates = pd.DataFrame(rates)
    # Set column names
    pd_rates.columns = ["Time", "Low", "High", "Open", "Close", "Volume"]
    # Convert Unix time to UTC
    pd_rates["Time"] = pd.to_datetime(pd_rates["Time"],unit='s')
    # Remove duplicates
    before = len(pd_rates)
    pd_rates.drop_duplicates(inplace=True)
    print("Dropped {} duplicate rows".format(before - len(pd_rates)))
    print(pd_rates.head())
    # Save to CSV
    fn = "./historic_data/{}_{}_{}.csv".format(product, granularity, len(pd_rates))
    pd_rates.to_csv(fn, index=False)
    return pd_rates


In [4]:
save_rates('BTC-USD', granularity=60 * 5, num_candles=10000)

From: 2019-10-26T03:16:00, To: 2019-10-27T04:16:00 - 300 points
From: 2019-10-27T04:16:00, To: 2019-10-28T05:16:00 - 300 points
From: 2019-10-28T05:16:00, To: 2019-10-29T06:16:00 - 300 points
From: 2019-10-29T06:16:00, To: 2019-10-30T07:16:00 - 300 points
From: 2019-10-30T07:16:00, To: 2019-10-31T08:16:00 - 300 points
From: 2019-10-31T08:16:00, To: 2019-11-01T09:16:00 - 285 points
From: 2019-11-01T09:16:00, To: 2019-11-02T10:16:00 - 300 points
From: 2019-11-02T10:16:00, To: 2019-11-03T11:16:00 - 300 points
From: 2019-11-03T11:16:00, To: 2019-11-04T12:16:00 - 300 points
From: 2019-11-04T12:16:00, To: 2019-11-05T13:16:00 - 300 points
From: 2019-11-05T13:16:00, To: 2019-11-06T14:16:00 - 300 points
From: 2019-11-06T14:16:00, To: 2019-11-07T15:16:00 - 300 points
From: 2019-11-07T15:16:00, To: 2019-11-08T16:16:00 - 300 points
From: 2019-11-08T16:16:00, To: 2019-11-09T17:16:00 - 300 points
From: 2019-11-09T17:16:00, To: 2019-11-10T18:16:00 - 300 points
From: 2019-11-10T18:16:00, To: 2019-11-1

Time      Low     High     Open    Close      Volume
0    2019-10-27 04:15:00  9102.33  9126.78  9126.78  9126.00   64.758102
1    2019-10-27 04:10:00  9126.77  9150.08  9138.38  9129.62   14.681644
2    2019-10-27 04:05:00  9136.82  9147.00  9143.78  9138.38   12.730305
3    2019-10-27 04:00:00  9143.61  9167.67  9165.85  9146.75   16.303569
4    2019-10-27 03:55:00  9165.87  9181.09  9169.97  9165.88   20.006930
...                  ...      ...      ...      ...      ...         ...
9980 2019-11-29 12:40:00  7579.90  7592.00  7583.26  7591.05   16.380793
9981 2019-11-29 12:35:00  7570.41  7599.38  7572.73  7583.26   12.293334
9982 2019-11-29 12:30:00  7565.33  7598.18  7594.67  7572.73   21.240514
9983 2019-11-29 12:25:00  7584.97  7635.05  7627.65  7594.66  250.681425
9984 2019-11-29 12:20:00  7490.77  7630.00  7490.77  7626.49  506.839460

[9985 rows x 6 columns]

In [5]:
all_prods = client.get_products()
for prod in all_prods:
    prod_str = prod['id']
    if ('USD' in prod_str and 'USDC' not in prod_str) or ('EUR' in prod_str):
        print("Downloading data for {}".format(prod_str))
        save_rates(prod_str, granularity=60 * 5, num_candles=200000)

From: 2016-12-16T08:52:00, To: 2016-12-17T09:52:00 - 0 points
From: 2016-12-17T09:52:00, To: 2016-12-18T10:52:00 - 0 points
From: 2016-12-18T10:52:00, To: 2016-12-19T11:52:00 - 0 points
From: 2016-12-19T11:52:00, To: 2016-12-20T12:52:00 - 0 points
From: 2016-12-20T12:52:00, To: 2016-12-21T13:52:00 - 0 points
From: 2016-12-21T13:52:00, To: 2016-12-22T14:52:00 - 0 points
From: 2016-12-22T14:52:00, To: 2016-12-23T15:52:00 - 0 points
From: 2016-12-23T15:52:00, To: 2016-12-24T16:52:00 - 0 points
From: 2016-12-24T16:52:00, To: 2016-12-25T17:52:00 - 0 points
From: 2016-12-25T17:52:00, To: 2016-12-26T18:52:00 - 0 points
From: 2016-12-26T18:52:00, To: 2016-12-27T19:52:00 - 0 points
From: 2016-12-27T19:52:00, To: 2016-12-28T20:52:00 - 0 points
From: 2016-12-28T20:52:00, To: 2016-12-29T21:52:00 - 0 points
From: 2016-12-29T21:52:00, To: 2016-12-30T22:52:00 - 0 points
From: 2016-12-30T22:52:00, To: 2016-12-31T23:52:00 - 0 points
From: 2016-12-31T23:52:00, To: 2017-01-02T00:52:00 - 0 points
From: 20